# What is the likelihood a U.S. House seat will flip to Democratic?

In [1]:
import pandas as pd
import time
# Input is a start time and an end time
# Output is a print statement giving the time elapsed
def time_elapsed(start, end):
    # Compute time elapsed in seconds
    total_time_seconds = end-start 
    if total_time_seconds < 60:
        print("Total time elapsed =",total_time_seconds, "seconds")
    else:
        # In minutes 
        total_time_minutes = total_time_seconds/60 
        if total_time_minutes < 60: 
            print("Total time elapsed =", total_time_minutes, "minutes") 
        else: 
            # In hours
            total_time_hours = total_time_minutes/60 
            # Print the time elapsed in hours
            print("Total time elapsed =", total_time_hours, "hours")

## Data cleaning

In [2]:
with open("Original\\data\\codebook-us-house-1976–2020.md") as f:
    lines = f.readlines()
    
# Quick-and-dirty way to get the column names, since they seem to be misaligned
columns = [l.replace("###", "").strip() for l in lines if l.startswith("###")]
#print(columns) # for debugging

df = pd.read_csv("Original\\data\\1976-2020-house.tab", sep="\t", header=None)

# Choose whatever columns you want to keep; I made these choices with manual inspection
df = df.iloc[:, [0, 1, 7, 12, 15, 16]]
# Sets the name of each column to the ith column in the metadata
df.columns = [columns[i] for i in [0, 1, 7, 11, 14, 15]]
#print(df.columns) # for debugging
df # for debugging

,year,state,district,party,candidatevotes,totalvotes
0,1976,ALABAMA,1,DEMOCRAT,58906,157170
1,1976,ALABAMA,1,REPUBLICAN,98257,157170
2,1976,ALABAMA,1,NaN,7,157170
3,1976,ALABAMA,2,DEMOCRAT,66288,156362
4,1976,ALABAMA,2,REPUBLICAN,90069,156362
...,...,...,...,...,...,...
31098,2020,WYOMING,0,DEMOCRAT,66576,278503
31099,2020,WYOMING,0,NaN,1274,278503
31100,2020,WYOMING,0,LIBERTARIAN,10154,278503
31101,2020,WYOMING,0,NaN,6337,278503


In [3]:
# Only keep the elections using Census 2010 districts
df = df.loc[df.year >= 2012].copy() # copy eliminates a warning
df # for debugging

,year,state,district,party,candidatevotes,totalvotes
24053,2012,ALABAMA,1,REPUBLICAN,196374,200676
24054,2012,ALABAMA,1,NaN,4302,200676
24055,2012,ALABAMA,2,REPUBLICAN,180591,283953
24056,2012,ALABAMA,2,DEMOCRAT,103092,283953
24057,2012,ALABAMA,2,NaN,270,283953
...,...,...,...,...,...,...
31098,2020,WYOMING,0,DEMOCRAT,66576,278503
31099,2020,WYOMING,0,NaN,1274,278503
31100,2020,WYOMING,0,LIBERTARIAN,10154,278503
31101,2020,WYOMING,0,NaN,6337,278503


In [4]:
# Add a column for the winner of each race
df["winner"] =  0
#print(df.winner.values.tolist()) # for debugging

# Make lists of the states and years
states = []
[states.append(state) for state in df.state.values.tolist() if state not in states]
years = []
[years.append(year) for year in df.year.values.tolist() if year not in years]

# Function to get the districts for each state
def districts(state):
    district_numbers_repeats = df.loc[(df.year==2012) & (df.state==state), "district"].values.tolist()
    district_numbers = []
    [district_numbers.append(num) for num in district_numbers_repeats if num not in district_numbers]
    return district_numbers

for state in states:
    #print(state) # for debugging
    #start = time.time() # for debugging
    district_numbers = districts(state)
    #end = time.time() # for debugging
    #time_elapsed(start, end) # for debugging
    for year in years:
        #print(year) # for debugging
        #start = time.time() # for debugging
        for district in district_numbers:
            max_votes = df.loc[(df.year==year) & (df.state==state) & (df.district == district), "candidatevotes"].max()
            #print("Max votes for", state, year,"district", district, "is", max_votes) # for debugging
            df.loc[(df.year==year) & (df.state==state) & (df.district==district) & (df.candidatevotes==max_votes), "winner"] = 1
        #end = time.time() # for debugging
        #time_elapsed(start, end) # for debugging 
df.sample(10).sort_index() # for debugging      

,year,state,district,party,candidatevotes,totalvotes,winner
24694,2012,MINNESOTA,4,INDEPENDENCE,21135,347991,0
25521,2014,ARIZONA,7,INDEPENDENT,3496,72454,0
27068,2016,FLORIDA,5,REPUBLICAN,108325,302874,0
27662,2016,NEW YORK,4,NaN,356,343179,0
28268,2016,WISCONSIN,8,NaN,16,363780,0
28714,2018,KENTUCKY,6,INDEPENDENT,1011,302888,0
29084,2018,NEW YORK,12,DEMOCRAT,205858,251877,1
29208,2018,NORTH CAROLINA,8,DEMOCRAT,114119,255521,0
29237,2018,OHIO,3,DEMOCRAT,181575,246677,1
30020,2020,ILLINOIS,10,WRITE-IN,18,316874,0


In [5]:
# Need to include elections in which there was no Democrat running before 
# executing the next block
for state in states:
    district_numbers = districts(state)
    for year in years:
        for district in district_numbers:
            if "DEMOCRAT" not in df.loc[(df.year==year) & (df.state==state) & (df.district==district), "party"].values.tolist():
                total_votes_rows = df.loc[(df.year==year) & (df.state==state) & (df.district==district), "totalvotes"]
                total_votes = total_votes_rows.iloc[0]
                #print(total_votes) # for debugging
                new_row = {"year":year, "state":state, "district":district, "party":"DEMOCRAT", "candidatevotes":0, "totalvotes":total_votes, "winner":0}
                df.loc[len(df)] = new_row
df = df.sort_values(by=["year", "state", "district"])
df # for debugging

,year,state,district,party,candidatevotes,totalvotes,winner
24053,2012,ALABAMA,1,REPUBLICAN,196374,200676,1
24054,2012,ALABAMA,1,NaN,4302,200676,0
7050,2012,ALABAMA,1,DEMOCRAT,0,200676,0
24055,2012,ALABAMA,2,REPUBLICAN,180591,283953,1
24056,2012,ALABAMA,2,DEMOCRAT,103092,283953,0
...,...,...,...,...,...,...,...
31098,2020,WYOMING,0,DEMOCRAT,66576,278503,0
31099,2020,WYOMING,0,NaN,1274,278503,0
31100,2020,WYOMING,0,LIBERTARIAN,10154,278503,0
31101,2020,WYOMING,0,NaN,6337,278503,0


In [6]:
# Only look at the results for Democrats
df = df.loc[df.party == "DEMOCRAT"].copy()
df.sample(10).sort_index() # for debugging

,year,state,district,party,candidatevotes,totalvotes,winner
7053,2016,ALABAMA,1,DEMOCRAT,0,215893,0
25296,2012,TEXAS,18,DEMOCRAT,146223,194932,1
25852,2014,INDIANA,2,DEMOCRAT,55590,145200,0
26499,2014,PENNSYLVANIA,4,DEMOCRAT,50250,197340,0
26763,2014,VIRGINIA,11,DEMOCRAT,106780,187805,1
27546,2016,NEVADA,2,DEMOCRAT,115722,313336,0
28652,2018,INDIANA,5,DEMOCRAT,137142,317177,0
29218,2018,NORTH CAROLINA,12,DEMOCRAT,203974,279138,1
29468,2018,TEXAS,17,DEMOCRAT,98070,237351,0
31078,2020,WISCONSIN,3,DEMOCRAT,199870,389618,1


In [7]:
# Drop the total number of votes & D votes, and just keep the fraction of 
# D votes
df["fraction"] = df.loc[:, "candidatevotes"] / df.loc[:, "totalvotes"]
df = df.drop(columns=["candidatevotes", "totalvotes"])
df.reset_index(inplace=True, drop=True)
df.sample(10).sort_index() # for debugging

,year,state,district,party,winner,fraction
454,2014,ARIZONA,4,DEMOCRAT,0,0.257902
481,2014,CALIFORNIA,17,DEMOCRAT,0,0.482464
502,2014,CALIFORNIA,35,DEMOCRAT,1,0.634519
530,2014,CONNECTICUT,2,DEMOCRAT,1,0.576013
687,2014,NEBRASKA,3,DEMOCRAT,0,0.246124
701,2014,NEW JERSEY,8,DEMOCRAT,1,0.773536
773,2014,OREGON,3,DEMOCRAT,1,0.723289
1267,2016,TEXAS,7,DEMOCRAT,0,0.438264
1627,2018,NEW YORK,26,DEMOCRAT,1,0.680508
1881,2020,FLORIDA,4,DEMOCRAT,0,0.389003


In [8]:
# Add a column indicating whether the seat flipped to Democrat
df["flipped"] = ''
for i in range(len(years)):
    if i >= 1:
        for state in states:
            district_numbers = districts(state)
            for district in district_numbers:
                #print(years[i], state, district) # for debugging
                current_outcome = df.loc[(df.year==years[i]) & (df.state==state) & (df.district==district), "winner"].values[0]
                previous_outcome = df.loc[(df.year==years[i-1]) & (df.state==state) & (df.district==district), "winner"].values[0] 
                #print("In", years[i], "win was:", current_outcome) # for debugging
                #print("Last election, win was:", previous_outcome) # for debugging
                if (current_outcome == 1) & (previous_outcome ==0):
                    df.loc[(df.year==years[i]) & (df.state==state) & (df.district==district), "flipped"] = 1
                else: 
                    df.loc[(df.year==years[i]) & (df.state==state) & (df.district==district), "flipped"] = 0
                #print("Flipped =", df.loc[(df.year==years[i]) & (df.state==state) & (df.district==district), "flipped"].values[0]) # for debugging   
df.sample(10).sort_index() # for debugging                    

,year,state,district,party,winner,fraction,flipped
121,2012,GEORGIA,2,DEMOCRAT,1,0.637837,
766,2014,OKLAHOMA,1,DEMOCRAT,0,0.000000,0
1086,2016,MARYLAND,6,DEMOCRAT,1,0.559594,0
1270,2016,TEXAS,10,DEMOCRAT,0,0.384421,0
1360,2018,CALIFORNIA,4,DEMOCRAT,0,0.458685,0
1386,2018,CALIFORNIA,28,DEMOCRAT,1,0.783720,0
1707,2018,TEXAS,2,DEMOCRAT,0,0.455564,0
1993,2020,MASSACHUSETTS,8,DEMOCRAT,1,0.717204,0
2050,2020,NEW JERSEY,12,DEMOCRAT,1,0.656430,0
2101,2020,OHIO,7,DEMOCRAT,0,0.291745,0


In [9]:
## Data exploration

In [10]:
# Maybe we could make a similar table where the values are the flipped values
# and compare it to this one.

df_pivoted = pd.pivot_table(
    df,
    index=["state", "district", "party"],
    columns="year",
    values="fraction"
)
df_pivoted

year                             2012      2014      2016      2018      2020
state     district party                                                     
ALABAMA   1        DEMOCRAT  0.000000  0.317130  0.000000  0.367765  0.355387
          2        DEMOCRAT  0.363060  0.325641  0.405262  0.384259  0.346827
          3        DEMOCRAT  0.358270  0.337224  0.329320  0.362184  0.324593
          4        DEMOCRAT  0.259016  0.000000  0.000000  0.201291  0.176830
          5        DEMOCRAT  0.349380  0.000000  0.331578  0.388947  0.000000
...                               ...       ...       ...       ...       ...
WISCONSIN 5        DEMOCRAT  0.320502  0.304033  0.292897  0.379878  0.398330
          6        DEMOCRAT  0.377826  0.408691  0.372560  0.444637  0.407204
          7        DEMOCRAT  0.437969  0.394096  0.382705  0.385042  0.392140
          8        DEMOCRAT  0.439670  0.349408  0.372978  0.362757  0.357933
WYOMING   0        DEMOCRAT  0.229649  0.220873  0.291613  0.297662  0.239049

[436 rows x 5 columns]

In [11]:
# Only one row gets dropped with this code, but I don't know which one.
# Also, the year is the column name for the indexing.  Not sure how to fix that.

# Some districts didn't have elections in both years; drop those & reset the 
# index
df_final = df_pivoted.reset_index().dropna()
df_final

year,state,district,party,2012,2014,2016,2018,2020
0,ALABAMA,1,DEMOCRAT,0.000000,0.317130,0.000000,0.367765,0.355387
1,ALABAMA,2,DEMOCRAT,0.363060,0.325641,0.405262,0.384259,0.346827
2,ALABAMA,3,DEMOCRAT,0.358270,0.337224,0.329320,0.362184,0.324593
3,ALABAMA,4,DEMOCRAT,0.259016,0.000000,0.000000,0.201291,0.176830
4,ALABAMA,5,DEMOCRAT,0.349380,0.000000,0.331578,0.388947,0.000000
...,...,...,...,...,...,...,...,...
431,WISCONSIN,5,DEMOCRAT,0.320502,0.304033,0.292897,0.379878,0.398330
432,WISCONSIN,6,DEMOCRAT,0.377826,0.408691,0.372560,0.444637,0.407204
433,WISCONSIN,7,DEMOCRAT,0.437969,0.394096,0.382705,0.385042,0.392140
434,WISCONSIN,8,DEMOCRAT,0.439670,0.349408,0.372978,0.362757,0.357933


In [12]:
# It might be a better idea to get descriptive statistics over the years for 
# each district, then include in the table the flipped value.

# For example... what can we say about the average, etc. share of D votes in 
# each state?
df_final.groupby("state")[2018].describe()

,count,mean,std,min,25%,50%,75%,max
state,,,,,,,,
ALABAMA,7.0,0.427172,0.251598,0.201291,0.334947,0.367765,0.386603,0.978045
ALASKA,1.0,0.464971,NaN,0.464971,0.464971,0.464971,0.464971,0.464971
ARIZONA,9.0,0.532831,0.160023,0.305091,0.445340,0.538316,0.610856,0.856063
ARKANSAS,4.0,0.346132,0.076399,0.287744,0.306147,0.319287,0.359272,0.458211
CALIFORNIA,53.0,0.630940,0.155866,0.000000,0.522510,0.648555,0.741324,0.890755
COLORADO,7.0,0.529838,0.129406,0.393150,0.414677,0.541032,0.603594,0.738142
CONNECTICUT,5.0,0.586003,0.034686,0.527965,0.579906,0.603951,0.606095,0.612100
DELAWARE,1.0,0.644540,NaN,0.644540,0.644540,0.644540,0.644540,0.644540
FLORIDA,26.0,0.543940,0.219853,0.323544,0.395131,0.463375,0.583633,1.000000


## Model selection